This notebook mines for any recent `stock` news sources, and saves them to a file. This is run daily.

In [1]:
import os 
from finvizfinance.news import News
from finvizfinance.screener.overview import Overview
from finvizfinance.quote import finvizfinance
import pandas as pd 
import yfinance as yf
from datetime import timedelta, datetime
from investor_watch.constants import TICKER_PATH, STOCK_LIST_PATH
import time 

stock = 'AAPL'
stock_list = pd.read_csv(STOCK_LIST_PATH)
stock_list


,Ticker,Company,Sector,Industry,Country,Price,Market Cap (billions)
0,A,Agilent Technologies Inc,Healthcare,Diagnostics & Research,USA,147.36,42.09
1,AA,Alcoa Corp,Basic Materials,Aluminum,USA,39.53,10.21
2,AAL,American Airlines Group Inc,Industrials,Airlines,USA,18.27,12.01
3,AAON,AAON Inc,Industrials,Building Products & Equipment,USA,130.18,10.58
4,AAPL,Apple Inc,Technology,Consumer Electronics,USA,229.98,3458.41
...,...,...,...,...,...,...,...
672,ZBRA,Zebra Technologies Corp,Technology,Communication Equipment,USA,405.71,20.93
673,ZG,Zillow Group Inc,Communication Services,Internet Content & Information,USA,74.83,18.02
674,ZM,Zoom Communications Inc,Technology,Software - Application,USA,78.07,23.93
675,ZS,Zscaler Inc,Technology,Software - Infrastructure,USA,187.58,28.78


In [2]:
def get_news(ticker):       # sinks new news sources to csv files
    try:
        time.sleep(0.2)
        whitelist = ['Bloomberg', 'Reuters']
        news_df = finvizfinance(ticker).ticker_news()
        news_df = news_df[news_df['Source'].isin(whitelist)]              # whitelist

        file = f"{TICKER_PATH}/{ticker}.csv"
        last_modified = datetime.fromtimestamp(os.path.getmtime(file))
        news_df = news_df[(news_df['Date'] >= last_modified)].sort_values('Date')       # filter new news inbetween modified date and today

        print(f'Found {len(news_df)} new sources of ${ticker} ...')   
        return news_df

    except Exception as e:
        print(e)

news_df = get_news(stock)
news_df


Found 0 new sources of $AAPL ...


,Date,Title,Link,Source


In [ ]:
# write back to file if nessacary 
file = f"{TICKER_PATH}/{stock}.csv"
news_df.to_csv(file, mode='a', header=False, index=False)         # append enriched news back to local file storage